In [1]:
from wisdm import wisdm
import random
import numpy as np
import pandas as pd
from collections import Counter
import time
from scipy import stats

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [7]:
raw_df = pd.read_pickle('./datasets/all_raw_data.dataframe.pickle')

In [8]:
raw_df.head()

,user,class,timestamp,x-acc,y-acc,z-acc
0,1679,Walking,2013-06-06 07:07:49.556,0.294132,-0.635605,-0.226936
1,1679,Walking,2013-06-06 07:07:49.606,-0.499688,-0.604451,-0.226020
2,1679,Walking,2013-06-06 07:07:49.656,-2.178345,0.713491,0.372017
3,1679,Walking,2013-06-06 07:07:49.706,-2.797763,1.354899,-0.277638
4,1679,Walking,2013-06-06 07:07:49.756,-2.167961,-1.327716,-0.554971


In [9]:
data_df = pd.read_pickle('./datasets/compatible_transformed_df.pickle')

In [12]:
study_user_ids = data_df['user'].unique()

# How many users and how much time do each represent?

In [13]:
total_user_ids = data_df['user'].unique()
len(user_ids)

325

In [14]:
# constrain the dataframe to only users who were in Lockhart's final dataset
raw_study_df = raw_df[raw_df['user'].isin(study_user_ids)]

In [15]:
len(raw_study_df)

16642648

In [16]:
len(raw_df)

44171254

In [17]:
len(raw_df) - len(raw_study_df)

27528606

In [18]:
timespans = []

for user_id in study_user_ids:
    times = raw_study_df[raw_study_df['user'] == user_id]['timestamp']
    first = times.min()
    last = times.max()
    span = last - first
    timespans.append(span)

In [19]:
min(timespans)

Timedelta('0 days 00:00:19.903000')

In [20]:
max(timespans)

Timedelta('15980 days 05:16:40.785000')

In [21]:
sorted_ind = np.argsort(timespans)

In [23]:
timespans = np.array(timespans)

In [34]:
list(timespans[sorted_ind[-30:]])

[Timedelta('0 days 08:11:47.077000'),
 Timedelta('0 days 13:12:33.940000'),
 Timedelta('0 days 17:32:33'),
 Timedelta('1 days 05:24:07.957000'),
 Timedelta('1 days 09:49:00.516000'),
 Timedelta('1 days 11:41:44.425000'),
 Timedelta('1 days 22:16:02.231000'),
 Timedelta('2 days 06:37:52.425000'),
 Timedelta('2 days 17:22:12.904000'),
 Timedelta('4 days 15:36:54.881000'),
 Timedelta('5 days 05:22:24.186000'),
 Timedelta('8 days 12:27:41.989000'),
 Timedelta('11 days 16:01:00.792000'),
 Timedelta('14 days 18:02:51.225000'),
 Timedelta('20 days 20:35:04.613000'),
 Timedelta('32 days 01:59:00.318000'),
 Timedelta('35 days 15:43:41.877000'),
 Timedelta('83 days 20:53:16.070000'),
 Timedelta('83 days 22:41:32.015000'),
 Timedelta('93 days 06:12:59.622000'),
 Timedelta('98 days 18:26:58.174000'),
 Timedelta('119 days 20:14:14.716000'),
 Timedelta('119 days 20:48:46.848000'),
 NaT,
 Timedelta('120 days 02:14:54.050000'),
 Timedelta('175 days 17:33:47.528000'),
 Timedelta('15980 days 05:16:40.78

In [27]:
type(timespans[sorted_ind[-1]])

pandas._libs.tslib.NaTType

In [28]:
nat = np.datetime64("NaT")

In [33]:
timespans[sorted_ind[-1]] is nat

False

In [35]:
len(study_user_ids)

50

In [36]:
len(timespans)

50

In [37]:
study_user_ids = np.array(study_user_ids)

In [39]:
study_user_ids[sorted_ind[-12:-2]]

array(['1707', '1703', '1320', '1104', '194', '1809', '1117', '1277',
       '1319', '1097'], dtype=object)

In [41]:
raw_df['class'].unique()

array(['Walking', 'LyingDown', 'Standing', 'Sitting', 'Jogging', 'Stairs',
       'NoLabel'], dtype=object)

In [43]:
labeled_df = raw_study_df[raw_study_df['class'] != 'NoLabel']

In [68]:
n_times = []
for ind, user_id in enumerate(study_user_ids):
    print("Building Trace for user : %s" % user_id)
    try:
        label_times = labeled_df[labeled_df['user'] == user_id]['timestamp']
        first_time = min(label_times)
        normalized_times = [(t - first_time).seconds / 3600 for t in label_times]
        n_times.append(normalized_times)
    except ValueError as ve:
        print("skipping, not enough data")
        continue

Building Trace for user : 194
Building Trace for user : 998
Building Trace for user : 1097
skipping, not enough data
Building Trace for user : 1104
Building Trace for user : 1117
Building Trace for user : 1205
Building Trace for user : 1238
Building Trace for user : 1246
Building Trace for user : 1247
Building Trace for user : 1253
Building Trace for user : 1269
Building Trace for user : 1274
Building Trace for user : 1277
Building Trace for user : 1280
Building Trace for user : 1319
Building Trace for user : 1320
Building Trace for user : 1477
Building Trace for user : 1480
Building Trace for user : 1491
Building Trace for user : 1511
Building Trace for user : 1512
Building Trace for user : 1518
Building Trace for user : 1531
Building Trace for user : 1554
Building Trace for user : 1559
Building Trace for user : 1603
Building Trace for user : 1676
Building Trace for user : 1679
Building Trace for user : 1683
Building Trace for user : 1696
Building Trace for user : 1703
Building Trace 

In [53]:
import plotly.figure_factory as ff

In [71]:
raw_study_df.head()

,user,class,timestamp,x-acc,y-acc,z-acc
0,1679,Walking,2013-06-06 07:07:49.556,0.294132,-0.635605,-0.226936
1,1679,Walking,2013-06-06 07:07:49.606,-0.499688,-0.604451,-0.226020
2,1679,Walking,2013-06-06 07:07:49.656,-2.178345,0.713491,0.372017
3,1679,Walking,2013-06-06 07:07:49.706,-2.797763,1.354899,-0.277638
4,1679,Walking,2013-06-06 07:07:49.756,-2.167961,-1.327716,-0.554971


In [55]:
n_times[0][:10]

[Timedelta('0 days 00:00:00'),
 Timedelta('0 days 00:00:00.051000'),
 Timedelta('0 days 00:00:00.101000'),
 Timedelta('0 days 00:00:00.206000'),
 Timedelta('0 days 00:00:00.256000'),
 Timedelta('0 days 00:00:00.316000'),
 Timedelta('0 days 00:00:00.366000'),
 Timedelta('0 days 00:00:00.416000'),
 Timedelta('0 days 00:00:00.466000'),
 Timedelta('0 days 00:00:00.534000')]

In [61]:
td = n_times[0][30]

In [58]:
td.min

Timedelta('-106752 days +00:12:43.145224')

In [59]:
td.minutes

AttributeError: 'Timedelta' object has no attribute 'minutes'

In [62]:
td.seconds

2

In [63]:
3602 / 3600

1.0005555555555556